In [1]:
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense , Conv2D , MaxPooling2D , Flatten , Dropout , BatchNormalization
from keras.applications import ResNet152V2 , MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150

def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir,label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                new_img = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                resized_img = cv2.resize(new_img,(img_size,img_size))
                data.append([resized_img,class_num])
            except:
                continue
    return np.array(data)      
    

In [3]:
train = get_data('E:/Chest X-Ray Images/chest_xray/chest_xray/train')
test = get_data('E:/Chest X-Ray Images/chest_xray/chest_xray/test')
val = get_data('E:/Chest X-Ray Images/chest_xray/chest_xray/val')

C:\Users\Home\anaconda3\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [5]:
X_train = []
y_train = []

X_test = []
y_test = []

X_val = []
y_val = []

for img,label in train:
    X_train.append(img)
    y_train.append(label)

for img,label in test:
    X_test.append(img)
    y_test.append(label)

for img,label in val:
    X_val.append(img)
    y_val.append(label)


In [6]:
X_train = np.array(X_train)/255
X_test = np.array(X_test)/255
X_val = np.array(X_val)/255

X_train = X_train.reshape(-1,img_size,img_size,1)
X_test = X_test.reshape(-1,img_size,img_size,1)
X_val = X_val.reshape(-1,img_size,img_size,1)

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)


16

In [7]:
# With data augmentation to prevent overfitting and handling the imbalance in dataset

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [7]:
#CNN Model without BatchNormalization and Dropout

model = Sequential()
model.add(Conv2D(filters = 64,kernel_size = (3,3), strides = 1, padding='same', input_shape = (img_size,img_size,1), activation="relu"))
model.add(MaxPooling2D((2,2),strides = 2,padding='same'))

model.add(Conv2D(filters = 64,kernel_size = (3,3), strides = 1, padding='same', activation="relu"))
model.add(MaxPooling2D((2,2),strides = 2,padding='same'))


model.add(Conv2D(filters = 128,kernel_size = (3,3), strides = 1, padding='same', activation="relu"))
model.add(MaxPooling2D((2,2),strides = 2,padding='same'))

model.add(Conv2D(filters = 256,kernel_size = (3,3), strides = 1, padding='same', activation="relu"))
model.add(MaxPooling2D((2,2),strides = 2,padding='same'))

model.add(Flatten()),
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=1,activation="sigmoid"))

model.compile(
    optimizer="adam",
    loss = "binary_crossentropy",
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 19, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 256)       2

In [11]:
history = model.fit(datagen.flow(X_train,y_train, batch_size = 32) ,epochs = 10 , validation_data = datagen.flow(X_test, y_test))

Epoch 1/10
163/163 [==============================] - 396s 2s/step - loss: 0.1638 - accuracy: 0.9333 - val_loss: 0.3781 - val_accuracy: 0.8638
Epoch 2/10
163/163 [==============================] - 409s 3s/step - loss: 0.1584 - accuracy: 0.9362 - val_loss: 0.5906 - val_accuracy: 0.8237
Epoch 3/10
163/163 [==============================] - 439s 3s/step - loss: 0.1506 - accuracy: 0.9394 - val_loss: 0.6245 - val_accuracy: 0.8157
Epoch 4/10
163/163 [==============================] - 443s 3s/step - loss: 0.1451 - accuracy: 0.9421 - val_loss: 0.5492 - val_accuracy: 0.7804
Epoch 5/10
163/163 [==============================] - 388s 2s/step - loss: 0.1483 - accuracy: 0.9413 - val_loss: 0.5330 - val_accuracy: 0.8125
Epoch 6/10
163/163 [==============================] - 372s 2s/step - loss: 0.1375 - accuracy: 0.9469 - val_loss: 0.5826 - val_accuracy: 0.7965
Epoch 7/10
163/163 [==============================] - 372s 2s/step - loss: 0.1346 - accuracy: 0.9440 - val_loss: 0.5673 - val_accuracy: 0.7949

In [13]:
model.evaluate(X_test,y_test)

20/20 [==============================] - 12s 586ms/step - loss: 0.2606 - accuracy: 0.9199


[0.26064497232437134, 0.9198718070983887]

In [12]:
model.evaluate(X_val,y_val)

1/1 [==============================] - 1s 845ms/step - loss: 0.3510 - accuracy: 0.8125


[0.351010799407959, 0.8125]

In [13]:
#CNN Model with BatchNormalization and Dropout

model2 = Sequential([
    Conv2D(filters = 64,kernel_size = (3,3), strides = 1, padding='same', input_shape = (img_size,img_size,1), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
    Dropout(0.2),
    
    Conv2D(filters = 64,kernel_size = (3,3), strides = 1, padding='same', activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
    Dropout(0.2),
    
    Conv2D(filters = 128,kernel_size = (3,3), strides = 1, padding='same', activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
    Dropout(0.2),
    
    Conv2D(filters = 256,kernel_size = (3,3), strides = 1, padding='same', activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
    Dropout(0.2),
    
    Flatten(),
    Dense(units=256,activation='relu'),
    BatchNormalization(),
    Dense(units=1,activation='sigmoid')   
])

model2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


model2.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 150, 150, 64)      640       
_________________________________________________________________
batch_normalization_17 (Batc (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 75, 75, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 75, 75, 64)        36928     
_________________________________________________________________
batch_normalization_18 (Batc (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 38, 38, 64)       

In [15]:
history2 = model2.fit(datagen.flow(X_train,y_train,batch_size=32),epochs=5,validation_data=datagen.flow(X_test,y_test))

Epoch 1/5
163/163 [==============================] - 1069s 7s/step - loss: 0.2886 - accuracy: 0.8867 - val_loss: 19.0359 - val_accuracy: 0.6250
Epoch 2/5
163/163 [==============================] - 1057s 6s/step - loss: 0.2035 - accuracy: 0.9164 - val_loss: 9.6160 - val_accuracy: 0.6250
Epoch 3/5
163/163 [==============================] - 1102s 7s/step - loss: 0.1965 - accuracy: 0.9197 - val_loss: 3.6214 - val_accuracy: 0.6250
Epoch 4/5
163/163 [==============================] - 983s 6s/step - loss: 0.1838 - accuracy: 0.9294 - val_loss: 4.5761 - val_accuracy: 0.6250
Epoch 5/5
163/163 [==============================] - 848s 5s/step - loss: 0.1671 - accuracy: 0.9367 - val_loss: 1.7630 - val_accuracy: 0.4936


In [16]:
model2.evaluate(X_test,y_test)

20/20 [==============================] - 17s 734ms/step - loss: 1.1187 - accuracy: 0.5849


[1.118656039237976, 0.5849359035491943]

In [17]:
model2.evaluate(X_val,y_val)

1/1 [==============================] - 0s 397ms/step - loss: 1.3039 - accuracy: 0.4375


[1.303901195526123, 0.4375]